### Scrape ebay data and access required information using regex

##### Import Libraries

In [1]:
from bs4 import BeautifulSoup 
import requests
import re
import time

##### Load data from webpage

In [ ]:
def loadWebsiteData (url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        page = requests.get(url,headers=headers)
        # Create a beautifulsoup object 
        return BeautifulSoup(page.text, 'lxml')
    except :
        print("Error connecting to website")

In [ ]:
def save (data,fname):
    with open(fname, "w", encoding="utf-8") as file:
        file.write(data)

### Task 1 
Code that loads eBay's search result page containing sold "amazon gift card". Save the result to file. Name the file "amazon_gift_card_01.htm"

In [ ]:
soup = loadWebsiteData("https://www.ebay.com/sch/i.html?_nkw=amazon+gift+card&LH_Sold=1")

In [ ]:
save(str(soup),"amazon_gift_card_01.htm")

### Task 2
Loop through the search results and download the first 10 pages of search results. Save each of these pages to "amazon_gift_card_XX.htm" (XX = page number).

In [ ]:
url = "https://www.ebay.com/sch/i.html?_nkw=amazon+gift+card&LH_Sold=1&_pgn="
for i in range(1,11):
    pgnum = str(i) if i>=10 else "0"+str(i)
    save(str(loadWebsiteData(url+str(i))),"amazon_gift_card_"+pgnum+".htm")
    time.sleep(10)

### Task 3
Print to screen the title, price, and shipping price of each item.
Using RegEx, identify and print to screen gift cards that sold above face value. Compare a gift card’s value to its price + shipping (free shipping should be treated as 0).  If value < price + shipping, then a gift card sells above face value.

In [ ]:
abovefacevaluecount = 0
for i in range(1,11):
    pgnum = str(i) if i>=10 else "0"+str(i)
    name = "amazon_gift_card_"+pgnum+".htm"
    ## Part (c)
    HTMLFile = open(name, "rb")
    htmlfiledata = HTMLFile.read()
    soup = BeautifulSoup(htmlfiledata, 'lxml')
    items = soup.select("div.s-item__title--tag")
    actualprice=''
    price=''
    shipping=''
    for t in items:
        ## Part (d)
        if(t.nextSibling.find('span', attrs={'role': 'heading'})):
            title = t.nextSibling.find('span', attrs={'role': 'heading'}).text
            title = title.split("New Listing")[-1]
            print("Title : "+title)
            actualprice = "0" if len(re.findall("(?:[\$]{1}[,\d]+.?\d*)",title)) ==0 else re.findall("(?:[\$]{1}[,\d]+.?\d*)",title)[0]
        details = t.nextSibling.nextSibling
        if(details.find("span",attrs={'class':'POSITIVE'})):
            price = details.find("span",attrs={'class':'POSITIVE'}).text
            print("Price : "+price)
        if(details.find("span",attrs={'class':'s-item__shipping s-item__logisticsCost'})):
            shipping = details.find("span",attrs={'class':'s-item__shipping s-item__logisticsCost'}).text
            print("Shipping : "+shipping)
        actualprice = re.findall(r"\d+?.*?\d*", actualprice)[0]
        price = re.findall(r"\d+.*\d*",price)[0]
        shipping = "0" if len(re.findall(r"\d+.*\d*\W",shipping)) ==0 else re.findall(r"\d+.*\d*\W",shipping)[0] 
        morethanfacevalue = float(actualprice) < float(price)+float(shipping)
        ## Part (e)
        if(morethanfacevalue & (float(actualprice)!=0) ):
            print("This item is sold above face value!")
            print(actualprice +" < "+price +" + "+shipping)
            abovefacevaluecount += 1
        else:
            print("This item is not sold above face value :)")
        print("-----------------------------------")

### Task 4 - Sample Webscraping that requires login
Write code that automatically logs into the website fctables.com
And verifies that you have successfully logged in. Access https://www.fctables.com/tipster/my_bets/ Check whether the word “Wolfsburg” appears on the page.

In [5]:
try:
    URL = "https://www.fctables.com/user/login/"
    session_requests = requests.session()
    res = session_requests.post(URL, 
                                data = {
#                                     "login_username" : "sripriya",
#                                         "login_password" : "bayernmunich",
                                    "login_username" : "idislikebayernaswell",
                                        "login_password" : "gomanchesterunited",
                                        "user_remeber":"1",
                                        "login_action" : "1"},
                                timeout = 15
                               )
    print(res.status_code)
    cookies = session_requests.cookies.get_dict()
    print(cookies)
    time.sleep(5) 
    betURL = "https://www.fctables.com/tipster/my_bets/"
    page2 = session_requests.get(betURL,cookies=cookies)
        
    doc2 = BeautifulSoup(page2.content, 'html.parser')

    ## Search for text "Wolfsburg" on bets page
    print(bool(doc2.findAll(text = re.compile('Wolfsburg*'))))
    
except Exception as ex:
    print('error: ' + str(ex))

200
{'PHPSESSID': '3tuum1consj56cejdsvr4f3u02', 'auth_auto': '57458756d5af92a13a687253089f739e', 'user': '127026', 'user_logged': '1', 'user_login': 'idislikebayernaswell'}
True
